# Tflite Compatible log mel spectrogram calculator
- Built to ensure cross platform uniform calculation of spectrogram 
- Can process only one waveform at a time

In [2]:
!pip install librosa
import librosa
import tensorflow as tf

     |████████████████████████████████| 203 kB 3.1 MB/s eta 0:00:01
  Using cached joblib-1.0.1-py3-none-any.whl (303 kB)
     |████████████████████████████████| 323 kB 6.5 MB/s eta 0:00:01
     |████████████████████████████████| 23.8 MB 6.0 MB/s eta 0:00:01     |██████████████████████▌         | 16.8 MB 4.5 MB/s eta 0:00:02     |████████████████████████        | 17.8 MB 4.5 MB/s eta 0:00:02
     |████████████████████████████████| 377 kB 998 kB/s eta 0:00:01
     |████████████████████████████████| 3.4 MB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 28.5 MB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 26.3 MB 402 kB/s  eta 0:00:01     |████████████████████████        | 19.6 MB 10.1 MB/s eta 0:00:01     |██████████████████████████      | 21.4 MB 10.1 MB/s eta 0:00:01
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Created wheel for audioread: filename=audioread-2.1.9-py3-none-a

2021-09-14 21:17:07.480544: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-14 21:17:07.480575: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
%config Completer.use_jedi = False

In [10]:
import numpy as np
import tensorflow as tf
import math
import fractions
import datetime

LOG_MEL_MEAN = 1.4
LOG_MEL_STD = 1.184


import tensorflow as tf
import numpy as np
# import fractions
import math



def _dft_matrix(dft_length):
  """Calculate the full DFT matrix in numpy."""
  omega = (0 + 1j) * 2.0 * np.pi / float(dft_length)
  # Don't include 1/sqrt(N) scaling, tf.signal.rfft doesn't apply it.
  return np.exp(omega * np.outer(np.arange(dft_length), np.arange(dft_length)))


def _naive_rdft(signal_tensor, fft_length, padding='center'):
    """Implement real-input Fourier Transform by matmul."""
    # We are right-multiplying by the DFT matrix, and we are keeping
    # only the first half ("positive frequencies").
    # So discard the second half of rows, but transpose the array for
    # right-multiplication.
    # The DFT matrix is symmetric, so we could have done it more
    # directly, but this reflects our intention better.
    complex_dft_matrix_kept_values = _dft_matrix(fft_length)[:(fft_length // 2 + 1), :].transpose()
    real_dft_tensor = tf.constant(np.real(complex_dft_matrix_kept_values).astype(np.float32), name='real_dft_matrix')
    imag_dft_tensor = tf.constant(np.imag(complex_dft_matrix_kept_values).astype(np.float32), name='imaginary_dft_matrix')
    signal_frame_length = signal_tensor.shape[-1]#.value
    half_pad = (fft_length - signal_frame_length) // 2

    if padding == 'center':
        # Center-padding.
        pad_values = tf.concat([
            tf.zeros([tf.rank(signal_tensor) - 1, 2], tf.int32),
            [[half_pad, fft_length - signal_frame_length - half_pad]]
        ], axis=0)
    elif padding == 'right':
        # Right-padding.
        pad_values = tf.concat([
            tf.zeros([tf.rank(signal_tensor) - 1, 2], tf.int32),
            [[0, fft_length - signal_frame_length]]
        ], axis=0)

    padded_signal = tf.pad(signal_tensor, pad_values)
    
    result_real_part = tf.matmul(padded_signal, real_dft_tensor)
    result_imag_part = tf.matmul(padded_signal, imag_dft_tensor)
    
    return result_real_part, result_imag_part


def _fixed_frame(signal, frame_length, frame_step, first_axis=False):
    """tflite-compatible tf.signal.frame for fixed-size input.
    Args:
        signal: Tensor containing signal(s).
        frame_length: Number of samples to put in each frame.
        frame_step: Sample advance between successive frames.
        first_axis: If true, framing is applied to first axis of tensor; otherwise,
        it is applied to last axis.
    Returns:
        A new tensor where the last axis (or first, if first_axis) of input
        signal has been replaced by a (num_frames, frame_length) array of individual
        frames where each frame is drawn frame_step samples after the previous one.
    Raises:
        ValueError: if signal has an undefined axis length.  This routine only
        supports framing of signals whose shape is fixed at graph-build time.
    """
    signal_shape = signal.shape.as_list()
    
    if first_axis:
        length_samples = signal_shape[0]
    else:
        length_samples = signal_shape[-1]
    
    if length_samples <= 0:
        raise ValueError('fixed framing requires predefined constant signal length')
    
    num_frames = max(0, 1 + (length_samples - frame_length) // frame_step)
    
    if first_axis:
        inner_dimensions = signal_shape[1:]
        result_shape = [num_frames, frame_length] + inner_dimensions
        gather_axis = 0
    else:
        outer_dimensions = signal_shape[:-1]
        result_shape = outer_dimensions + [num_frames, frame_length]
        # Currently tflite's gather only supports axis==0, but that may still
        # work if we want the last of 1 axes.
        gather_axis = len(outer_dimensions)

    subframe_length = math.gcd(frame_length, frame_step)  # pylint: disable=deprecated-method
    subframes_per_frame = frame_length // subframe_length
    subframes_per_hop = frame_step // subframe_length
    num_subframes = length_samples // subframe_length

    if first_axis:
        trimmed_input_size = [num_subframes * subframe_length] + inner_dimensions
        subframe_shape = [num_subframes, subframe_length] + inner_dimensions
    else:
        trimmed_input_size = outer_dimensions + [num_subframes * subframe_length]
        subframe_shape = outer_dimensions + [num_subframes, subframe_length]
    subframes = tf.reshape(
        tf.slice(
            signal,
            begin=np.zeros(len(signal_shape), np.int32),
            size=trimmed_input_size), subframe_shape)

    # frame_selector is a [num_frames, subframes_per_frame] tensor
    # that indexes into the appropriate frame in subframes. For example:
    # [[0, 0, 0, 0], [2, 2, 2, 2], [4, 4, 4, 4]]
    frame_selector = np.reshape(np.arange(num_frames) * subframes_per_hop, [num_frames, 1])

    # subframe_selector is a [num_frames, subframes_per_frame] tensor
    # that indexes into the appropriate subframe within a frame. For example:
    # [[0, 1, 2, 3], [0, 1, 2, 3], [0, 1, 2, 3]]
    subframe_selector = np.reshape(np.arange(subframes_per_frame), [1, subframes_per_frame])

    # Adding the 2 selector tensors together produces a [num_frames,
    # subframes_per_frame] tensor of indices to use with tf.gather to select
    # subframes from subframes. We then reshape the inner-most subframes_per_frame
    # dimension to stitch the subframes together into frames. For example:
    # [[0, 1, 2, 3], [2, 3, 4, 5], [4, 5, 6, 7]].
    selector = frame_selector + subframe_selector
    frames = tf.reshape(tf.gather(subframes, selector.astype(np.int32), axis=gather_axis), result_shape)
    
    return frames


def _stft_tflite(signal, frame_length, frame_step, fft_length):
    """tflite-compatible implementation of tf.signal.stft.
    Compute the short-time Fourier transform of a 1D input while avoiding tf ops
    that are not currently supported in tflite (Rfft, Range, SplitV).
    fft_length must be fixed. A Hann window is of frame_length is always
    applied.
    Since fixed (precomputed) framing must be used, signal.shape[-1] must be a
    specific value (so "?"/None is not supported).
    Args:
        signal: 1D tensor containing the time-domain waveform to be transformed.
        frame_length: int, the number of points in each Fourier frame.
        frame_step: int, the number of samples to advance between successive frames.
        fft_length: int, the size of the Fourier transform to apply.
    Returns:
        Two (num_frames, fft_length) tensors containing the real and imaginary parts
        of the short-time Fourier transform of the input signal.
    """
    # Make the window be shape (1, frame_length) instead of just frame_length
    # in an effort to help the tflite broadcast logic.
    window = tf.reshape(
        tf.constant(
            (0.5 - 0.5 * np.cos(2 * np.pi * np.arange(0, 1.0, 1.0 / frame_length))
            ).astype(np.float32),
            name='window'), [1, frame_length])
    
    framed_signal = _fixed_frame(signal, frame_length, frame_step, first_axis=False)
    framed_signal *= window
    
    real_spectrogram, imag_spectrogram = _naive_rdft(framed_signal, fft_length)
    
    return real_spectrogram, imag_spectrogram


def _stft_magnitude_tflite(signals, frame_length, frame_step, fft_length):
    """Calculate spectrogram avoiding tflite incompatible ops."""
    real_stft, imag_stft = _stft_tflite(signals, frame_length, frame_step, fft_length)
    stft_magnitude = tf.sqrt(tf.add(real_stft * real_stft, imag_stft * imag_stft), name='magnitude_spectrogram')
    
    return stft_magnitude


def waveform_to_log_mel_spectrogram(waveform):
    """Compute log mel spectrogram of a 1-D waveform."""
    with tf.name_scope('log_mel_features'):
        # waveform has shape [<# samples>]

        window_length_samples = 400 #int(round(params.SAMPLE_RATE * params.STFT_WINDOW_SECONDS))
        hop_length_samples = 160    #int(round(params.SAMPLE_RATE * params.STFT_HOP_SECONDS))
        MEL_BANDS   = 64            #params.MEL_BANDS
        SAMPLE_RATE = 16000         #params.SAMPLE_RATE
        LOG_OFFSET  = 0.001         #params.LOG_OFFSET
        MEL_MIN_HZ  = 50            #params.MEL_MIN_HZ
        MEL_MAX_HZ  = 8000         #params.MEL_MAX_HZ
        fft_length  = 2 ** int(np.ceil(np.log(window_length_samples) / np.log(2.0)))
        num_spectrogram_bins = fft_length // 2 + 1
        magnitude_spectrogram = _stft_magnitude_tflite(
            signals=waveform, 
            frame_length=window_length_samples, 
            frame_step=hop_length_samples, 
            fft_length=fft_length)

        linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
            num_mel_bins=MEL_BANDS,
            num_spectrogram_bins=num_spectrogram_bins,
            sample_rate=SAMPLE_RATE,
            lower_edge_hertz=MEL_MIN_HZ,
            upper_edge_hertz=MEL_MAX_HZ)

        mel_spectrogram = tf.matmul(
            magnitude_spectrogram, linear_to_mel_weight_matrix)
        log_mel_spectrogram = (tf.math.log(
            mel_spectrogram + LOG_OFFSET) - LOG_MEL_MEAN)/LOG_MEL_STD
        # log_mel_spectrogram has shape [<# STFT frames>, MEL_BANDS]

        return log_mel_spectrogram

def build_log_mel_graph():
    inp = tf.keras.layers.Input(batch_shape=(None,16000))
    return tf.keras.Model(
        inputs=inp,
        outputs=waveform_to_log_mel_spectrogram(
            tf.squeeze(inp,axis=0)
        )
    )

In [7]:
!pip install matplotlib
import librosa
import librosa.display # for waveplots, spectograms, etc
import IPython.display as ipd # for playing files within pythonimport numpy as np
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

     |████████████████████████████████| 10.3 MB 6.4 MB/s eta 0:00:01    |████▍                           | 1.4 MB 2.4 MB/s eta 0:00:04
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
     |████████████████████████████████| 3.0 MB 5.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 7.1 MB/s eta 0:00:01


#### Converting log mel spectrogram calculation to tflite for optimization

In [11]:
from time import sleep, time
import tflite_runtime.interpreter as tflite

logmelcalc = build_log_mel_graph()
converter = tf.lite.TFLiteConverter.from_keras_model(logmelcalc)
logmelcalc_tflite = converter.convert()

logmelcalc_interpreter = tf.lite.Interpreter(model_content=logmelcalc_tflite)#allocate the tensors
logmelcalc_interpreter.allocate_tensors()

input_index = logmelcalc_interpreter.get_input_details()[0]["index"]
output_details = logmelcalc_interpreter.get_output_details()

s=time()
for i in range(1000):
    t = np.expand_dims(np.random.random(16000).astype("float32"),axis=0)
    logmelcalc_interpreter.set_tensor(input_index,t)
    logmelcalc_interpreter.invoke()
    output_data = logmelcalc_interpreter.get_tensor(output_details[0]['index'])
print("tflite optimized:",(time()-s)/1000)

s = time()
for i in range(1000):
    q=logmelcalc.predict(t)
print("No optimization:",(time()-s)/1000)


2021-09-14 21:21:14.597712: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmpm4_8yvtf/assets


2021-09-14 21:21:17.982235: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2021-09-14 21:21:17.982405: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2021-09-14 21:21:17.984882: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.012ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.

2021-09-14 21:21:18.062476: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:351] Ignored output_format.
2021-09-14 21:21:18.062517: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:354] Ignored drop_control_dependency.
2021-09-14 21:21:18.070170: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:210] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2021-09-14 21:21:1

tflite optimized: 0.0026256754398345946


2021-09-14 21:21:20.763816: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


No optimization: 0.048804286241531374


In [13]:
import pyaudio
import numpy as np
from time import time,sleep
# import cv2
from IPython.display import clear_output
from matplotlib import pyplot as plt
import IPython.display as ipd
import librosa.display

CHUNK = 4000
RATE = 16000

p=pyaudio.PyAudio()
stream=p.open(format=pyaudio.paInt16,channels=1,rate=RATE,input=True,
              frames_per_buffer=CHUNK)

inpAudio = np.zeros(RATE)

s = time()
for i in range(int(100*RATE/CHUNK)): #go for a few seconds
    #clear_output()
    
    data = np.frombuffer(stream.read(CHUNK),dtype=np.int16)
    inpAudio = np.append(inpAudio[CHUNK:],data)

    logmelcalc_interpreter.set_tensor(input_index,np.expand_dims(inpAudio/inpAudio.max(),axis=0).astype("float32"))
    logmelcalc_interpreter.invoke()
    output_data = logmelcalc_interpreter.get_tensor(output_details[0]['index'])
    #plt.imshow(output_data)
    #plt.show()
    sleep(0.24)
print(time()-s)

ALSA lib pcm_dsnoop.c:638:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1075:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
jack server is not running or cannot be started
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
jack server is not running or cannot be started
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
ALSA lib pcm

100.39566493034363


# Extracting BaseModel from Siamese Network

In [15]:
base_model = tf.keras.models.load_model("model-8-01-0.9551.h5").get_layer("basemodel")
temp = tf.keras.Model(inputs=base_model.inputs,outputs=base_model.outputs)
temp.summary()

ValueError: bad marshal data (unknown type code)

# Converting BaseModel to tflite for optimization
- Not reducing number of floating points

In [43]:
from time import sleep , time
import tflite_runtime.interpreter as tflite

converter = tf.lite.TFLiteConverter.from_keras_model(source_model)
base_model_tflite = converter.convert()

base_model_interpreter = tf.lite.Interpreter(model_content=base_model_tflite)#allocate the tensors
base_model_interpreter.allocate_tensors()

base_model_inp = base_model_interpreter.get_input_details()
base_model_out = base_model_interpreter.get_output_details()



print("Testing model speed")

temp = np.zeros((98,64))

s = time()
for i in range(100):
    base_model_interpreter.set_tensor(
        base_model_inp[0]["index"],
        np.expand_dims(np.random.random((98,64)),axis=(0,-1)).astype("float32")
    )

    base_model_interpreter.invoke()
    output_data = base_model_interpreter.get_tensor(base_model_out[0]["index"])
print((time()-s)/100)
print(output_data.shape)

INFO:tensorflow:Assets written to: /tmp/tmpt0n4dsqm/assets


INFO:tensorflow:Assets written to: /tmp/tmpt0n4dsqm/assets
/media/captain-america/7c8239eb-ca13-409c-8761-c865a3119962/Programming/UniversalHotwordDetector/.venv37/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
2021-09-01 14:52:32.193567: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2021-09-01 14:52:32.193708: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2021-09-01 14:52:32.205759: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.01ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.

2021-09-01 14:52:32.822179: W tensorfl

Testing model speed
0.005234899520874023
(1, 128)


In [39]:
from pprint import pprint

base_model_interpreter = tf.lite.Interpreter(model_content=base_model_tflite)#allocate the tensors
base_model_interpreter.allocate_tensors()

base_model_inp = base_model_interpreter.get_input_details()
base_model_out = base_model_interpreter.get_output_details()

pprint(base_model_inp)

pprint(base_model_out)

base_model_interpreter.set_tensor(
        base_model_inp[0]["index"],
        np.expand_dims(np.random.random((98,64)),axis=(0,-1)).astype("float32")
    )

base_model_interpreter.invoke()
output_data = base_model_interpreter.get_tensor(base_model_out[0]["index"])
output_data.shape

[{'dtype': <class 'numpy.float32'>,
  'index': 0,
  'name': 'input_1',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
                              'scales': array([], dtype=float32),
                              'zero_points': array([], dtype=int32)},
  'shape': array([ 1, 98, 64,  1], dtype=int32),
  'shape_signature': array([-1, 98, 64,  1], dtype=int32),
  'sparsity_parameters': {}}]
[{'dtype': <class 'numpy.float32'>,
  'index': 258,
  'name': 'Identity',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
                              'scales': array([], dtype=float32),
                              'zero_points': array([], dtype=int32)},
  'shape': array([  1, 128], dtype=int32),
  'shape_signature': array([ -1, 128], dtype=int32),
  'sparsity_parameters': {}}]


(1, 128)

In [29]:
t=base_model.predict(np.expand_dims(np.random.random((98,64)),axis=(0,-1)))

In [30]:
t.shape

(1, 128)